In [1]:
import json
import os
from collections import defaultdict
import pickle
import json
from tqdm.auto import tqdm
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

/home/brandon/Projet_TER/-Interactive-Emergency-Response-Dashboard-Haute-Garonne-Mike/-Interactive-Emergency-Response-Dashboard-Haute-Garonne-Mike/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device is cuda


In [2]:
import os
print(os.getcwd())
os.chdir('/home/brandon/Projet_TER/-Interactive-Emergency-Response-Dashboard-Haute-Garonne-Mike/-Interactive-Emergency-Response-Dashboard-Haute-Garonne-Mike/Scripts')
print(os.getcwd())

/home/brandon/Projet_TER/-Interactive-Emergency-Response-Dashboard-Haute-Garonne-Mike/-Interactive-Emergency-Response-Dashboard-Haute-Garonne-Mike/Scripts
/home/brandon/Projet_TER/-Interactive-Emergency-Response-Dashboard-Haute-Garonne-Mike/-Interactive-Emergency-Response-Dashboard-Haute-Garonne-Mike/Scripts


In [3]:
import pandas as pd
import numpy as np

print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)

Pandas version: 2.2.3
NumPy version: 2.2.6


# Train

In [4]:
%run train.py --lr 0.0025 --layers 1024 --num_timesteps 1000 --is_y_cond --save_as 'dqn_test' 

Index(['Month_sin', 'Month_cos', 'Hour_sin', 'Hour_cos', 'Day_sin', 'Day_cos',
       'Coord X', 'Coord Y', 'Duration', 'Incident', 'Month', 'Day', 'Hour'],
      dtype='object')
Index(['Month_sin', 'Month_cos', 'Hour_sin', 'Hour_cos', 'Day_sin', 'Day_cos',
       'Coord X', 'Coord Y', 'Duration', 'Incident'],
      dtype='object')
K [0]
features 9
{'d_in': np.int64(9), 'is_y_cond': True, 'num_classes': 56, 'rtdl_params': {'d_layers': [1024, 1024], 'dropout': 0.0}, 'dim_t': 128}
mlp
label embedding Embedding(56, 128)
diffusion ready
Model trained


# Sample

In [5]:
for i in range(10): # génère i datasets
    print("iteration", i)
    filename = f"df_fake_{i}.pkl"
    
    process = subprocess.Popen(
        ["python", "sample.py", 
         "--load_as", 'dqn_test', 
         "--save_sample_as", filename, 
         "--os_factor", "3", 
         "--to_keep", "40", 
         "--value_span", "100"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1  # pour forcer le flush ligne par ligne
    )

    # process.wait() # évite le lazy

    line_count = 0
    for line in process.stdout:
        line_count += 1
        if line_count % 50 == 0:
            clear_output(wait=True)
    
        print(line.strip())

    print(f"--- {filename} done---\n")

iteration 0
os factor: 3 sample batch size: 8192 variability: 0.02
[0]
features 9
{'d_in': np.int64(9), 'is_y_cond': True, 'num_classes': 56, 'rtdl_params': {'d_layers': [1024, 1024], 'dropout': 0.0}, 'dim_t': 128}
mlp
label embedding Embedding(56, 128)
params loaded
diffusion ready
Index(['area_name', 'count', 'new_samples', 'perc.', 'delta'], dtype='object')
new samples OK
(53088, 14)
Index(['Coord X', 'Coord Y', 'Duration', 'Day', 'Month', 'Hour', 'Incident',
'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos', 'Hour_sin', 'Hour_cos',
'area_name'],
dtype='object')
dataset sampled
--- df_fake_0.pkl done---

iteration 1
os factor: 3 sample batch size: 8192 variability: 0.02
[0]
features 9
{'d_in': np.int64(9), 'is_y_cond': True, 'num_classes': 56, 'rtdl_params': {'d_layers': [1024, 1024], 'dropout': 0.0}, 'dim_t': 128}
mlp
label embedding Embedding(56, 128)
params loaded
diffusion ready
Index(['area_name', 'count', 'new_samples', 'perc.', 'delta'], dtype='object')
new samples OK
(53088, 14

# Generate environment

In [ ]:
# lister les fichiers samplés à concaténer

In [6]:
%%time

%run generate_environment.py --sample_list df_fake_0.pkl df_fake_1.pkl df_fake_2.pkl df_fake_3.pkl df_fake_4.pkl df_fake_5.pkl df_fake_6.pkl df_fake_7.pkl df_fake_8.pkl df_fake_9.pkl --save_as "df_pc_fake_10y.pkl"

is probabilistic departure False


FileNotFoundError: [Errno 2] No such file or directory: 'sorties_2018.csv'

CPU times: user 426 ms, sys: 30.4 ms, total: 457 ms
Wall time: 525 ms


# Generate metrics

In [6]:
dic_tarif_zero = {'v_sent': 0,
                  'v_sent_full': 0,
            'v_degraded':0,
            'cancelled':0, #cancel departure
            'function_not_found':0,
            'v1_not_sent_from_1st_station':0,
            'v_not_found_in_last_station':0,
            # 'ff_sent':0,
            # 'ff_skill_mean':0,
            'z1_sent': 0,
            'skill_lvl':0
            }

In [7]:
os.chdir('./Reward_weights')
#os.chdir('./Reward_weights')

metrics = list(dic_tarif_zero.keys())

suffix = "r100_cf3"

for m in metrics:
    
    dic_tarif = dic_tarif_zero.copy()
    if m in ['v_sent', 'v_sent_full', 'skill_lvl']:
        dic_tarif[m] = 100
    else:
        dic_tarif[m] = -100
        # dic_tarif["v_sent_full"] = 5

    with open(f"rw_{m}_{suffix}.json", "w") as f:
        json.dump(dic_tarif, f)

os.chdir("../")

In [8]:
os.chdir('./Reward_weights')

with open(f"rw_skill_lvl_{suffix}.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

{'v_sent': 0,
 'v_sent_full': 0,
 'v_degraded': 0,
 'cancelled': 0,
 'function_not_found': 0,
 'v1_not_sent_from_1st_station': 0,
 'v_not_found_in_last_station': 0,
 'z1_sent': 0,
 'skill_lvl': 100}

# Simulation

In [9]:
from IPython.display import clear_output

m = 'z1_sent'
suffix = "r100_cf3"

cmd = [
    "python3", "-u", "simulation_start.py",
    "--reward_weights", f"rw_{m}_{suffix}.json",
    "--dataset", "df_pc_real.pkl",
    "--start", "1",
    "--end", "53088",
    "--constraint_factor_veh", "3",
    "--constraint_factor_ff", "1",
    "--save_metrics_as", f"sim_metrics_{m}_{suffix}",
    "--is_best"
]

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
for line in process.stdout:
    line_count += 1
    if line_count % 50 == 0:
        clear_output(wait=True)

    print(line.strip())

49500 v_out: 28 rwd_mean: 0.00 per ff: -0.18 act: 257768 per act.: -0.12414 v_not_found_ls: 384 deg: 1008 z1_sent: 2893
49600 v_out: 13 rwd_mean: 0.00 per ff: -0.18 act: 258286 per act.: -0.12389 v_not_found_ls: 384 deg: 1010 z1_sent: 2895
49700 v_out: 3 rwd_mean: 0.00 per ff: -0.17 act: 258831 per act.: -0.12363 v_not_found_ls: 385 deg: 1011 z1_sent: 2900
49800 v_out: 14 rwd_mean: 0.00 per ff: -0.17 act: 259491 per act.: -0.12332 v_not_found_ls: 385 deg: 1018 z1_sent: 2901
49900 v_out: 11 rwd_mean: 0.00 per ff: -0.17 act: 259935 per act.: -0.12311 v_not_found_ls: 385 deg: 1018 z1_sent: 2901
50000 v_out: 2 rwd_mean: 0.00 per ff: -0.17 act: 260550 per act.: -0.12282 v_not_found_ls: 385 deg: 1019 z1_sent: 2901
50100 v_out: 9 rwd_mean: 0.00 per ff: -0.17 act: 261026 per act.: -0.12336 v_not_found_ls: 386 deg: 1019 z1_sent: 2906
50200 v_out: 3 rwd_mean: 0.00 per ff: -0.17 act: 261481 per act.: -0.12314 v_not_found_ls: 386 deg: 1019 z1_sent: 2906
50300 v_out: 14 rwd_mean: 0.00 per ff: -0.17

# Agent params

In [10]:
action_size = 80
train_seed = 41

os.chdir("./Data")


hyper_params = {"state_size" : (action_size + 2) *40,
                "action_size" : action_size,
                "layer_type" : "ff", # noisy else ff
                "layer_size" : 1024,
                "num_layers" : 8,
                "use_batchnorm" : True,
                "n_steps" : 3, #5, 1, 50, 64
                "batch_size" : 256, #512, 256, 64, 32, 16, 8
                "buffer_size" : 200000, #100000, 128, 5000, 10000, 65 buffer_size > batch_size

                "update_every" : 32, #1, 200, 500, 64, 32 Q updates
                "per" : 1, # 0 for curiosity > 0, else 1, 2
                "rdm" : 0, # only if not per
                "munchausen" : 1, #1
                "curiosity" : 0, #Adds intrinsic curiosity to the extrinsic reward. 0 - only reward/ no curiosity, 
                                                                                    #1 - reward and curiosity, 
                                                                                    #2 - only curiosity
                "curiosity_size" : 1024,
                "lr" : 1e-3, #1e-3, 5e-4, 5e-3
                "lr_dec" : 1, #0, 1, 2, 3
                "entropy_tau" : 0.03, #0.03, 0.05 idem #-  Munch param
                "entropy_tau_coeff" : 1e-2, #1e-2 #-  Munch param
                "lo" : -1, #-  Munch param
                "alpha" : 0.9, #-  Munch param
                "gamma" : 0.99,
                "tau" : 0.005, #1e-2, 5e-3
                "N" : 32,# Number of quantiles 32, 64
                "entropy_coeff" : 0.001,
                "device" : str(device),
                "seed" : train_seed}


json.dump(hyper_params, open("hyper_params.json", "w"))

os.chdir("../")

In [11]:
m = 'z1_sent'
suffix = "r100_cf3"

cmd = [
    "python3", "-u", "agent_run.py",
    "--model_name", f"agent_{m}_{suffix}",
    "--hyper_params", "hyper_params.json",
    "--reward_weights", f"rw_{m}_{suffix}.json",
    "--dataset", "df_pc_fake_10y.pkl",
    "--start", "1",
    "--end", "530880",
    "--constraint_factor_veh", "3",
    "--constraint_factor_ff", "1",
    "--save_metrics_as", f"agent_metrics_{m}_{suffix}",
    "--train"
]

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
for line in process.stdout:
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

528500 v_out: 3 rwd_mean: 0.00 per ff: -0.03 act: 2944936 per act.: -0.02326 v_not_found_ls: 5211 deg: 27795 z1_sent: 4448 eps: 0.12 lr: 0.00000
528600 v_out: 24 rwd_mean: 0.00 per ff: -0.03 act: 2945572 per act.: -0.02326 v_not_found_ls: 5211 deg: 27803 z1_sent: 4448 eps: 0.12 lr: 0.00000
528700 v_out: 16 rwd_mean: 0.00 per ff: -0.03 act: 2946167 per act.: -0.02325 v_not_found_ls: 5212 deg: 27806 z1_sent: 4448 eps: 0.12 lr: 0.00000
528800 v_out: 10 rwd_mean: 0.00 per ff: -0.03 act: 2946798 per act.: -0.02325 v_not_found_ls: 5214 deg: 27808 z1_sent: 4448 eps: 0.12 lr: 0.00000
step 2947456 current lr : 6.346455595343308e-07
528900 v_out: 17 rwd_mean: 0.00 per ff: -0.03 act: 2947613 per act.: -0.02324 v_not_found_ls: 5216 deg: 27826 z1_sent: 4448 eps: 0.12 lr: 0.00000
529000 v_out: 15 rwd_mean: 0.00 per ff: -0.03 act: 2948322 per act.: -0.02323 v_not_found_ls: 5217 deg: 27843 z1_sent: 4448 eps: 0.12 lr: 0.00000
529100 v_out: 4 rwd_mean: 0.00 per ff: -0.03 act: 2948855 per act.: -0.02323 